<a href="https://colab.research.google.com/github/Suresh045/TNSDC/blob/main/RF_Grid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#importing the Libraies
import numpy as np
import pandas as pd

In [2]:
# Reading the Dataset
dataset = pd.read_csv('Social_Network_Ads.csv')

In [3]:
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [4]:
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [5]:
dataset=pd.get_dummies(dataset,dtype=int,drop_first=True)

In [6]:
dataset

,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,1
1,15810944,35,20000,0,1
2,15668575,26,43000,0,0
3,15603246,27,57000,0,0
4,15804002,19,76000,0,1
...,...,...,...,...,...
395,15691863,46,41000,1,0
396,15706071,51,23000,1,1
397,15654296,50,20000,1,0
398,15755018,36,33000,0,1


In [7]:
indep=dataset[['Age', 'EstimatedSalary','Gender_Male']]
dep=dataset['Purchased']

In [8]:
#split into training set and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(indep, dep, test_size = 0.20, random_state = 0)


In [9]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [10]:
from sklearn.ensemble import RandomForestClassifier

In [11]:
#https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter

In [13]:
from sklearn.model_selection import GridSearchCV

# Define the parameter grid
# Create a parameter grid
param_grid = {
    'n_estimators': [100, 150, 200],        # Number of trees
    'criterion': ['gini', 'entropy'],       # Split quality measure
    'max_depth': [None, 10, 20],            # Tree depth
    'min_samples_split': [2, 5, 10],        # Min samples to split a node
    'max_features': ['auto', 'sqrt', 'log2']  # Features to consider for best split
}

grid = GridSearchCV(RandomForestClassifier(), param_grid, refit = True, verbose = 3,n_jobs=-1,scoring='f1_weighted')

# fitting the model for grid search
grid.fit(X_train, y_train)

Fitting 5 folds for each of 162 candidates, totalling 810 fits


/usr/local/lib/python3.12/dist-packages/sklearn/model_selection/_validation.py:528: FitFailedWarning: 
270 fits failed out of a total of 810.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
270 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/sklearn/model_selection/_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.12/dist-packages/sklearn/base.py", line 1382, in wrapper
    estimator._validate_params()
  File "/usr/local/lib/python3.12/dist-packages/sklearn/base.py", line 436, in _validate_params
    validate_parameter_constraints(
  File "/usr/local/lib/python3.12/dist-packages/sklearn/util

GridSearchCV(estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [None, 10, 20],
                         'max_features': ['auto', 'sqrt', 'log2'],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [100, 150, 200]},
             scoring='f1_weighted', verbose=3)

In [14]:
re=grid.cv_results_
grid_predictions = grid.predict(X_test)

In [15]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, grid_predictions)

In [16]:
# print classification report
from sklearn.metrics import classification_report
clf_report = classification_report(y_test, grid_predictions)

In [17]:
from sklearn.metrics import f1_score
f1_macro=f1_score(y_test,grid_predictions,average='weighted')
print("The f1_macro value for best parameter {}:".format(grid.best_params_),f1_macro)

The f1_macro value for best parameter {'criterion': 'entropy', 'max_depth': None, 'max_features': 'sqrt', 'min_samples_split': 10, 'n_estimators': 150}: 0.950648360030511


In [18]:
print("The confusion Matrix:\n",cm)

The confusion Matrix:
 [[55  3]
 [ 1 21]]


In [19]:
print("The report:\n",clf_report)

The report:
               precision    recall  f1-score   support

           0       0.98      0.95      0.96        58
           1       0.88      0.95      0.91        22

    accuracy                           0.95        80
   macro avg       0.93      0.95      0.94        80
weighted avg       0.95      0.95      0.95        80



In [20]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,grid.predict_proba(X_test)[:,1])

np.float64(0.9788401253918495)

In [21]:
# Assuming 'grid' is your GridSearchCV object that has been fitted to the data
best_model = grid.best_estimator_

In [22]:
import pickle
filename="RF_G.sav"
pickle.dump(best_model,open(filename,'wb'))

In [23]:
table=pd.DataFrame.from_dict(re)

In [24]:
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_depth,param_max_features,param_min_samples_split,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.001811,0.000862,0.000000,0.000000,gini,None,auto,2,100,"{'criterion': 'gini', 'max_depth': None, 'max_...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,109
1,0.001296,0.000542,0.000000,0.000000,gini,None,auto,2,150,"{'criterion': 'gini', 'max_depth': None, 'max_...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,109
2,0.001025,0.000047,0.000000,0.000000,gini,None,auto,2,200,"{'criterion': 'gini', 'max_depth': None, 'max_...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,109
3,0.001904,0.001648,0.000000,0.000000,gini,None,auto,5,100,"{'criterion': 'gini', 'max_depth': None, 'max_...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,109
4,0.000989,0.000024,0.000000,0.000000,gini,None,auto,5,150,"{'criterion': 'gini', 'max_depth': None, 'max_...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,109
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,0.426527,0.016545,0.025614,0.001269,entropy,20,log2,5,150,"{'criterion': 'entropy', 'max_depth': 20, 'max...",0.860542,0.85992,0.907389,0.907389,0.952916,0.897631,0.034769,56
158,0.579459,0.012860,0.031197,0.000575,entropy,20,log2,5,200,"{'criterion': 'entropy', 'max_depth': 20, 'max...",0.859841,0.87500,0.891711,0.907389,0.952916,0.897371,0.032021,66
159,0.287032,0.016349,0.019056,0.001350,entropy,20,log2,10,100,"{'criterion': 'entropy', 'max_depth': 20, 'max...",0.875759,0.85992,0.907389,0.907389,0.984436,0.906978,0.042870,5
160,0.421895,0.008758,0.024862,0.000323,entropy,20,log2,10,150,"{'criterion': 'entropy', 'max_depth': 20, 'max...",0.860542,0.85992,0.891711,0.907389,0.984436,0.900800,0.045635,36
